In [25]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt 
import re
import demoji
# demoji.download_codes()
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
import seaborn as sns
import emoji



%matplotlib inline

In [8]:
# conda install nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/anweshcr7/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anweshcr7/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
# pip install emoji

     |████████████████████████████████| 51kB 5.3MB/s eta 0:00:011
  Created wheel for emoji: filename=emoji-0.5.4-cp37-none-any.whl size=42177 sha256=422892a1468ae4635f9dd536d951ae845f45ce978e686e49d2985bb15125e65c
  Stored in directory: /Users/anweshcr7/Library/Caches/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
Note: you may need to restart the kernel to use updated packages.


In [9]:
# some additional imports
# from autocorrect import speller
stopwordsList = stopwords.words('english')
# ended up lemmatizing instead of stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [10]:
def parse_dataset(fp):
    '''
    Loads the dataset file with label-tweet on each line and parses the dataset.
    :param fp: filepath of dataset
    :return:
        corpus: list of tweet strings of each tweet.
        y: list of labels
    '''
    y = []
    corpus = []
    with open(fp, 'rt') as data_in:
        for line in data_in:
            if not line.lower().startswith("tweet index"): # discard first line if it contains metadata
                line = line.rstrip() # remove trailing whitespace
                label = int(line.split("\t")[1])
                tweet = line.split("\t")[2]
                y.append(label)
                corpus.append(tweet)

    return corpus, y

In [11]:
data, y_label = parse_dataset('SemEval2018-T3-train-taskA_emoji.txt')
data_df = pd.DataFrame(np.array(data).reshape(3834,1), columns = ['tweet'])
data_df['label'] = np.array(y_label).reshape(3834,1)
data_df.loc[1][0]

"@mrdahl87 We are rumored to have talked to Erv's agent... and the Angels asked about Ed Escobar... that's hardly nothing    ;)"

In [13]:
def preprocess(tweet):

    clean_tweet = clean(tweet.lower())
    return clean_tweet

def preprocess(tweet):
#     Remove URL
    tweet = re.sub(r'http\S+', '', tweet)
#     remove hashtag (symbols) and punctuations
    punctuations = '''!()-![]{};:+'"\,<>./?@#$%^&*_~'''
    tweet = ''.join([i for i in tweet if not i in punctuations])
#     reomve emoji
    tweet = demoji.replace(tweet)
#     correct spellings if any
# Lemmatize over Stemming
    tweet = " ".join([wordnet_lemmatizer.lemmatize(word) for word in tweet.split()])
# Gives clean_tweet but now lets tokenize
# tokenize and remove stop-words
    tweet = word_tokenize(tweet)
    return ' '.join([word for word in tweet if word not in stopwordsList])

In [41]:
# tweet = "@mrdahl87 We are rumored to have talked to Erv's agent... and the Angels asked about Ed Escobar... that's hardly nothing "
# tweet = " ".join([wordnet_lemmatizer.lemmatize(word) for word in tweet.split()])
# wordnet_lemmatizer.lemmatize('talked')

'talked'

In [38]:
def replace_hashtag_emoji(tweet):
    tweet = re.sub(r'\B#','', tweet)
    tweet = emoji.demojize(tweet, delimiters=("", ""))
    return tweet

# replace_hashtag_emoji('anwesh #hello 🔥 ')

In [34]:
# Some feature engineering before pre-processing (might be useful!)
def preprocess_and_engineer(df):
    df['eng_tweet'] = data_df['tweet'].apply(lambda x: replace_hashtag_emoji(x)) #creating a new column
#     df['emoji'] = data_df['tweet'].apply(lambda x: demoji.findall(x)) #create a new emoji column
    df['clean_tweet'] = df['eng_tweet'].apply(lambda x: preprocess(x))
#     del df['tweet']
    cols = ['clean_tweet','label']
    return df[cols]

In [35]:
# train_irony = data_df[ data_df['label'] == 1]
# train_irony = train_irony['tweet']
# train_normal = data_df[ data_df['label'] == 0]
# train_normal = train_normal['tweet']

# def wordcloud_draw(data, color = 'black'):
#     words = ' '.join(data)
#     cleaned_word = " ".join([word for word in words.split()
#                             if 'http' not in word
#                                 and not word.startswith('@')
#                                 and not word.startswith('#')
#                                 and word != 'RT'
#                             ])
#     wordcloud = WordCloud(stopwords=STOPWORDS,
#                       background_color=color,
#                       width=2500,
#                       height=2000
#                      ).generate(cleaned_word)
#     plt.figure(1,figsize=(13, 13))
#     plt.imshow(wordcloud)
#     plt.axis('off')
#     plt.show()
    
# print("Positive words")
# wordcloud_draw(train_irony,'white')
# print("Negative words")
# wordcloud_draw(train_normal)
# # Doesn't seem to make a lot of difference!

In [36]:
clean_data_df = preprocess_and_engineer(data_df)
clean_data_df.head()

,clean_tweet,label
0,Sweet United Nations video Just time Christmas...,1
1,mrdahl87 We rumored talked Ervs agent Angels a...,1
2,Hey Nice see MinnesotaND Winter Weather,1
3,3 episode left Im dying,0
4,I cant breathe wa chosen notable quote year an...,1


In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Returns a victorizer fitted with the training data
def vectorized_features(df):
    vectorizer = TfidfVectorizer(stop_words='english', sublinear_tf=True, strip_accents='unicode',
                                 analyzer='word', norm=None, ngram_range = (1,3))
    # tokenize and build vocab
    vectorizer.fit(df['clean_tweet'])
    return vectorizer

In [138]:
tweet_vectorizer = vectorized_features(clean_data_df)
# tweet_vector = tweet_vectorizer.transform(clean_data_df['clean_tweet'])
# tweet_vector

In [139]:
test_data, test_label = parse_dataset('SemEval2018-T3_gold_test_taskA_emoji.txt')
test_df = pd.DataFrame(np.array(test_data).reshape(784,1), columns = ['tweet'])
test_df['label'] = np.array(test_label).reshape(784,1)
test_df = preprocess_and_engineer(test_df)
test_df.head()

,clean_tweet,label
0,Sweet United Nations video Just time Christmas...,0
1,mrdahl87 We rumored talked Ervs agent Angels a...,1
2,Hey Nice see MinnesotaND Winter Weather,0
3,3 episode left Im dying,0
4,I cant breathe wa chosen notable quote year an...,1


In [140]:
test_vector = tweet_vectorizer.transform(test_df['clean_tweet'])
y_test = test_df['label']

In [144]:
# Training a Naive Bayes Implementation
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix


gnb = GaussianNB()
score = []
# kfold = KFold(n_splits=10, random_state=1, shuffle=True)
# for train, test in kfold.split(clean_data_df):
#     train_tf_idf = tweet_vectorizer.fit_transform(clean_data_df['clean_tweet'][train])
#     gnb.fit(train_tf_idf.todense(), clean_data_df['label'][train])
#     test_tf_idf = tweet_vectorizer.transform(clean_data_df['clean_tweet'][test])
#     y_pred = gnb.predict(test_tf_idf.todense())
#     score.append(metrics.accuracy_score(clean_data_df['label'][test], y_pred))
train_tf_idf = tweet_vectorizer.fit_transform(clean_data_df['clean_tweet'])
gnb.fit(train_tf_idf.todense(), clean_data_df['label'])

# from statistics import mean 

# print(mean(score))

GaussianNB(priors=None, var_smoothing=1e-09)

In [145]:
y_pred = gnb.predict(test_vector.toarray())
print(metrics.classification_report(y_test, y_pred, digits = 4))

              precision    recall  f1-score   support

           0     0.6347    0.5032    0.5613       473
           1     0.4254    0.5595    0.4833       311

    accuracy                         0.5255       784
   macro avg     0.5300    0.5313    0.5223       784
weighted avg     0.5517    0.5255    0.5304       784



In [146]:
def plotCM(cm):
    ax = plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, fmt="d"); #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(['Irony', 'Non-Irony']); ax.yaxis.set_ticklabels(['Irony', 'Non-Irony']);

In [ ]:
%config InlineBackend.figure_format ='retina'

nb_cm = confusion_matrix(y_test, y_pred)
plotCM(nb_cm)
